# MobileNet

MobileNet是一种为移动设备设计的神经网络架构。它由Google的研究团队开发并于2017年首次推出。MobileNet的主要目标是提供高性能、低延迟的图像分类和目标检测，适用于智能手机、平板电脑和其他资源受限的设备。

MobileNet通过使用深度可分离卷积来实现这一点，这是标准卷积的一种更高效的替代方法。深度可分离卷积将计算分解为两个独立的步骤：深度卷积（depthwise convolution）和逐点卷积（pointwise convolution）。这大大减少了参数和计算复杂度，使MobileNet能够在移动设备上高效运行。


MobileNet V1 的论文：https://arxiv.org/abs/1704.04861

<div class="alert alert-info">
**摘要**：我们提出了一类名为 MobileNet 的高效模型，专为移动和嵌入式视觉应用设计。MobileNet 基于一种简化的架构，使用深度可分离卷积构建轻量级深度神经网络。我们引入了两个简单的全局超参数，可以在延迟和准确性之间进行高效权衡。这些超参数允许模型构建者根据问题的约束选择适合的模型大小。我们进行了大量的实验，研究了资源和准确性之间的权衡，并显示出在 ImageNet 分类任务中，相比其他流行模型的优越表现。随后，我们展示了 MobileNet 在包括目标检测、细粒度分类、人脸属性识别和大规模地理定位等广泛应用中的有效性。。
</div>

## MobileNet 后续的版本

* MobileNet V2 中主要引用了 Inverted Residuals 和 Linear Bottleneck
* MobileNet V3 使用了 NAS 自动优化网络结构，引用了 Sequeeze-and-Excitation 模块，增强了通道间特征的相关性，并使用了新的激活函数。

## MobileNet中的卷积类型

![MobileNetV1 中的深度可分离卷积结构与传统的卷积结构的对比](assets/MobileNet_depthwise-separable-conv.png)

图一：MobileNetV1 中的深度可分离卷积结构（右边）与传统的卷积结构（左边）的对比

通过将常规卷积层替换为这些深度可分离卷积和逐点卷积，MobileNet在保持高准确率的同时，最大限度地减少了计算开销，使其非常适合移动设备和其他资源受限的平台。MobileNet使用了两种卷积类型：

### 深度可分离卷积

在传统的卷积层中，每个滤波器同时应用于所有输入通道。深度可分离卷积将这一过程分为两个步骤：深度卷积（depthwise convolution）和逐点卷积（pointwise convolution）。

此步骤对输入图像的每个通道（单一颜色或特征）分别使用小滤波器（通常为3x3）进行卷积。此步骤的输出与输入尺寸相同，但通道数量减少。

### 逐点可分离卷积

这种卷积使用一个单一的滤波器（通常为1x1），作用于输入和输出层的所有通道。它的参数比常规卷积更少，可以被视为全连接层的替代方法，非常适合计算资源有限的移动设备。

在深度卷积之后，此步骤通过另一个1x1卷积层将先前步骤的过滤输出组合起来。该操作有效地聚合了深度卷积学习的特征，将其简化为一组更小的特征，从而在保留重要信息的同时减少了整体复杂性。

## 为什么使用这些卷积而不是常规卷积？

为了更好地理解，以下是简化的解释：

### 常规卷积：大且全面的滤波器

想象有一个大而厚的滤波器（像是有多层的海绵）。这个滤波器应用于整个图像，同时处理图像的所有部分及其特征（如颜色）。这需要大量的计算和一个大的滤波器（存储空间）。

### 深度可分离卷积：轻量的两步处理

MobileNet将该过程简化为两个更小、更简单的步骤：

* **步骤1 - 深度卷积**： 首先，对每个图像特征分别使用一个薄滤波器（类似于海绵的单层），比如分别处理每种颜色。这减少了计算工作量，因为每个滤波器都较小且简单。
* **步骤2 - 逐点卷积**： 然后，使用另一个小滤波器（如一个小点）将这些特征重新组合。这个步骤类似于总结前面滤波器找到的信息。

### 这些步骤的意义

MobileNet用这两个更小的步骤替代了一个大步骤，相当于对常规卷积中的工作进行了轻量化。特别适合计算能力较弱的设备，如智能手机。

通过较小的滤波器，MobileNet不需要太多的存储空间。就像需要一个较小的盒子来存放所有工具，使其更适合小型设备。

## 1x1 卷积如何不同于普通卷积

### 普通卷积

* 普通卷积使用较大的核（如3x3或5x5）一次查看图像中的一组像素。这就像观察图片中的一个小块，以了解场景的一部分。
* 这种卷积通过分析像素的邻接关系来理解特征，例如边缘、角落和纹理。


### 1x1 卷积

* 1x1卷积一次仅查看一个像素，它不尝试理解相邻像素的排列。
* 尽管只查看一个像素，但它考虑来自不同通道的信息（例如彩色图像中的RGB通道）。它将这些通道组合以创建该像素的新版本。
* 1x1卷积可以增加或减少通道数，这意味着它既可以简化信息（通过减少通道），也可以创建更复杂的信息（通过增加通道）。

### 关键差异

* 关注区域： 普通卷积分析一组像素来理解模式，而1x1卷积专注于单个像素，结合不同通道的信息。
* 目的： 普通卷积用于检测图像中的模式和特征，而1x1卷积主要用于调整通道深度，以便在后续的神经网络层中更高效地处理信息。

## 倒置残差和线性瓶颈 

### 倒置残差（Inverted Residuals）

传统的残差结构（如 ResNet）通常采用“降维-卷积-升维”的方式，即先通过 1×1 卷积降低通道数，再通过 3×3 卷积提取特征，最后使用 1×1 卷积恢复通道数，形成一个“瓶颈”结构。倒置残差则反其道而行之，采用“升维-卷积-降维”的方式。

这种结构的优势在于：1）减少计算量：通过在高维空间中进行深度卷积，降低了计算复杂度。2）提升特征表达能力：在高维空间中提取特征，有助于捕获更多的信息。

### 线性瓶颈（Linear Bottlenecks）

在传统的卷积神经网络中，激活函数（如 ReLU）通常用于引入非线性。然而，研究发现，在低维空间中应用非线性激活可能导致信息丢失。为此，线性瓶颈提出在降维操作后，不使用非线性激活函数，即：在降维后的 1×1 卷积层中，移除激活函数，保持线性映射。这样做的目的是避免信息丢失：在低维空间中，非线性激活可能导致特征信息的丢失，线性映射可以保留更多的原始信息。


![MobileV2 中的线性瓶颈块与 ResNet 中的残差块的对比。](assets/MobileNet_inverted_residual.png)

图2: MobileV2 中的倒置的残差块与 ResNet 中的残差块的对比。

## ReLU 的作用是什么？

在训练过程中可能会遇到一些问题。我们先解释这些问题，然后再解释ReLU的作用。

### 梯度消失问题

在深度神经网络中，特别是在反向传播过程中，可能会出现梯度消失问题。当梯度（用于更新网络权重）在网络层中逐层传递时变得非常小时，就会出现这种情况。如果这些梯度变得过小，它们会“消失”，使网络难以有效地学习和调整权重。

由于ReLU在正值时具有线性、非饱和形式（当输入为正时，它简单地输出输入值），这确保了梯度不会变得过小，从而实现更快的学习和更有效的权重调整。

### 非线性

如果没有非线性，无论神经网络有多少层，它都将作为一个线性模型，无法学习复杂的模式。

像ReLU这样的非线性函数使神经网络能够捕捉并学习数据中的复杂关系。

## 基于 Pytorch 的从零实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 深度可分离卷积块的实现

MobileNet 中的核心 Layer 是由一个 Depthwise Conv 和一个 Pointwise Conv 组合形成的模块。其中 Depthwise Conv 保持特征图的通道数不变，而 Pointwise Conv 则负责对通信进行融合和调整。

In [2]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.depthwise = nn.Conv2d(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            groups=in_channels,
        )
        self.pointwise = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, stride=1, padding=0
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

### MobileNetV1 完整实现

MobileNet 的骨干部分由 14 个 Layer 构成，除了第一个 Layer，其余都是 DepthwiseSeparableConv。我们也可以将其分为 6 个 Stage：

* Stage 1: 只由一个 `conv1` 构成，通道数是 32，输出的 FeatureMap 大小减半
* Stage 2: 由 `dw_conv2`、`dw_conv3` 组成，输出通道数是 128，输出的 FeatureMap 大小减半
* Stage 3: 由 `dw_conv4`、`dw_conv5` 组成，输出通道数是 256，输出的 FeatureMap 大小减半
* Stage 4: 由 `dw_conv6`、`dw_conv7` 组成，输出通道数是 512 FeatureMap 大小减半
* Stage 5: 由 `dw_conv8` ~ `dw_conv13` 组成，输出通道数是 1024 FeatureMap 大小减半
* Stage 6: 由 `dw_conv14` 组成，输出通道数是 1024 FeatureMap 的尺寸保持不变

最后是一个 AvgPooling 和 一个 FC层的分类头。

In [3]:
class MobileNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)

        # MobileNet主体
        self.dw_conv2 = DepthwiseSeparableConv(32, 64, 1)
        self.dw_conv3 = DepthwiseSeparableConv(64, 128, 2)

        self.dw_conv4 = DepthwiseSeparableConv(128, 128, 1)
        self.dw_conv5 = DepthwiseSeparableConv(128, 256, 2)

        self.dw_conv6 = DepthwiseSeparableConv(256, 256, 1)
        self.dw_conv7 = DepthwiseSeparableConv(256, 512, 2)

        self.dw_conv8 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv9 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv10 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv11 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv12 = DepthwiseSeparableConv(512, 512, 1)
        self.dw_conv13 = DepthwiseSeparableConv(512, 1024, 2)

        self.dw_conv14 = DepthwiseSeparableConv(1024, 1024, 1)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = self.dw_conv2(x)
        x = F.relu(x)
        x = self.dw_conv3(x)
        x = F.relu(x)
        x = self.dw_conv4(x)
        x = F.relu(x)
        x = self.dw_conv5(x)
        x = F.relu(x)
        x = self.dw_conv6(x)
        x = F.relu(x)
        x = self.dw_conv7(x)
        x = F.relu(x)

        x = self.dw_conv8(x)
        x = F.relu(x)
        x = self.dw_conv9(x)
        x = F.relu(x)
        x = self.dw_conv10(x)
        x = F.relu(x)
        x = self.dw_conv11(x)
        x = F.relu(x)
        x = self.dw_conv12(x)
        x = F.relu(x)

        x = self.dw_conv13(x)
        x = F.relu(x)
        x = self.dw_conv14(x)
        x = F.relu(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [5]:
mobilenet = MobileNetV1(num_classes=1000)

x = torch.randn(1, 3, 224, 224)
out = mobilenet(x)
print(f"output shape of the MobileNet: {out.shape}")

output shape of the MobileNet: torch.Size([1, 1000])
